# An introduction to vector search
In this notebook, we're going to learn about vector search. Let's first install some libraries that we're going to use.

In [1]:
!pip install numpy scikit-learn plotly pandas

  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/26/5d/1e13b597ed8e54803e9ac6ded18c04cd35d8cbc49016778ec50c4ca9e9d5/plotly-5.16.1-py2.py3-none-any.whl.metadata
  Using cached plotly-5.16.1-py2.py3-none-any.whl.metadata (7.0 kB)
Using cached plotly-5.16.1-py2.py3-none-any.whl (15.6 MB)


## Creating vectors
Let's create some vectors to work with.

In [2]:
import numpy as np

In [3]:
vectors = np.random.randint(low=1, high=100, size=(20, 2), dtype='int32')
vectors[:5]

array([[88, 61],
       [68, 18],
       [37, 76],
       [54, 66],
       [54, 69]], dtype=int32)

In [4]:
import plotly.graph_objects as go

In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=vectors[:, 0], y=vectors[:, 1], mode='markers', marker=dict(size=8, color='blue'), name="Data"))
fig.update_layout(template="plotly_white", margin=dict(t=50, b=50, l=50, r=50), xaxis=dict(range=[0, 100]),  yaxis=dict(range=[0, 100]))
fig

## Finding nearest neighbours
Next, we're going to use sk-learn to find the nearest neighbour for a search vector.

In [6]:
search_vector = np.array([[10,10]])

In [7]:
fig.add_trace(go.Scatter(x=search_vector[:, 0], y=search_vector[:, 1], mode='markers', marker=dict(size=8, color='red'), name="Search Vector"))
fig

In [237]:
from sklearn.neighbors import NearestNeighbors

In [238]:
nn = NearestNeighbors(algorithm="brute", metric="minkowski")
nbrs = nn.fit(vectors)

In [239]:
distances, indices = nbrs.kneighbors(search_vector, n_neighbors=3)
print("Distances:", distances)
print("Indices:", indices)

Distances: [[ 3.16227766 22.56102835 25.3179778 ]]
Indices: [[ 2  3 17]]


In [240]:
for id, distance in zip(indices[0], distances[0]):
    print("Vector:", vectors[id])
    print("Distance:", distance)
    print("")

Vector: [11  7]
Distance: 3.1622776601683795

Vector: [32 15]
Distance: 22.561028345356956

Vector: [35  6]
Distance: 25.317977802344327



## More and bigger vectors
Let's have a look what happens when we use more and bigger vectors.

In [241]:
dimensions = 64
number_of_vectors = 5_000_000    
vectors = np.random.randint(low=1, high=100_000, size=(number_of_vectors, dimensions), dtype='int32')

In [242]:
brute = NearestNeighbors(algorithm="brute", metric="minkowski")
brute_nbrs = brute.fit(vectors)

In [243]:
kd_tree = NearestNeighbors(algorithm="kd_tree", metric="minkowski")
kd_tree_nbrs = kd_tree.fit(vectors)

In [244]:
ball_tree = NearestNeighbors(algorithm="ball_tree", metric="minkowski")
ball_tree_nbrs = ball_tree.fit(vectors)

In [245]:
search_vector = np.random.randint(1, 100_000, (1, dimensions), dtype='int32')
search_vector

array([[65774, 23719, 68122, 10998, 79071, 75469, 13656, 15312, 14018,
        66184, 12354, 66707, 99846, 59051, 82681, 59893, 78115,   165,
        87223, 50965, 93095, 89420, 16036,  1326, 48602, 51671, 28783,
        94460, 96277, 95185, 90372, 57151, 47895, 46204, 22162, 22573,
        20396, 67073, 19945, 25104, 24036, 61182, 39333, 46068, 44337,
        36640, 22742, 38246, 66721, 92639,  4828, 79387, 61985, 69682,
        29707, 51998, 24593, 75937,  1487, 86128, 99027, 65618, 40376,
        15094]], dtype=int32)

In [246]:
import time
def nearest_neighbours(algorithm, number_of_neighbours):
    start = time.time()
    distances, indices = algorithm.kneighbors(search_vector, n_neighbors=number_of_neighbours)
    end = time.time()

    print(f"Time: {end - start:.4f} seconds")

    print("Results:")
    return pd.DataFrame({
        "id": indices[0],
        "vector": [vectors[id] for id in indices[0]],
        "distance": distances[0]
    })

In [247]:
nearest_neighbours(brute_nbrs, 3)

Time: 1.2142 seconds
Results:


,id,vector,distance
0,7351,"[89790, 33042, 86264, 2645, 52425, 62478, 2676...",199643.101777
1,3115426,"[84484, 62107, 6116, 32108, 74041, 79194, 9738...",210536.966317
2,3288864,"[78224, 35131, 49818, 53602, 97015, 71875, 191...",216861.849132


In [248]:
nearest_neighbours(kd_tree_nbrs, 3)

Time: 0.9414 seconds
Results:


,id,vector,distance
0,7351,"[89790, 33042, 86264, 2645, 52425, 62478, 2676...",199643.101777
1,3115426,"[84484, 62107, 6116, 32108, 74041, 79194, 9738...",210536.966317
2,3288864,"[78224, 35131, 49818, 53602, 97015, 71875, 191...",216861.849132


In [249]:
nearest_neighbours(ball_tree_nbrs, 3)

Time: 0.8338 seconds
Results:


,id,vector,distance
0,7351,"[89790, 33042, 86264, 2645, 52425, 62478, 2676...",199643.101777
1,3115426,"[84484, 62107, 6116, 32108, 74041, 79194, 9738...",210536.966317
2,3288864,"[78224, 35131, 49818, 53602, 97015, 71875, 191...",216861.849132
